In [1]:
from google.cloud import bigquery

import pandas as pd
import numpy as np
import datetime
import os
import json
from haversine import haversine
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/docs/merkle-gdyr-prod-9783607c77a1.json"

zip_centers=json.load(open("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json"))
client = bigquery.Client()

print(datetime.datetime.now())
print(os.getcwd())

2020-10-26 18:41:06.789954
/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/Selections


In [12]:
excel_TA=pd.ExcelFile("/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/TA_creation/output_2020-10-23/GY_final_TA_updated_JL_2020-10-23.xlsx")
df_TA=excel_TA.parse("view_by_TA",dtype=str)
df_store=excel_TA.parse("view_by_store",dtype=str)


In [13]:
df_TA['store_count']=df_TA['store_count'].astype(int)
df_TA['count_Coor_store_in_P']=df_TA['count_Coor_store_in_P'].astype(int)
df_TA['count_Noncoor_store_in_P']=df_TA['count_Noncoor_store_in_P'].astype(int)
df_TA['store_count']=df_TA['store_count'].astype(int)
df_TA=df_TA[df_TA['count_Coor_store_in_P']>df_TA['count_Noncoor_store_in_P']]
df_TA=df_TA[(df_TA['store_count']>=2) & (df_TA['store_count']<=5)]
print(df_TA.shape,df_TA['store_count'].sum())
df_TA=df_TA.reset_index()
del df_TA['index']

print(df_TA.shape)
df_TA_HI=df_TA[df_TA['state_list'].apply(lambda x: "HI" in eval(str(x)))]
df_TA=df_TA[df_TA['state_list'].apply(lambda x: "HI" not in eval(str(x)))]
print(df_TA.shape)

(30, 14) 89
(30, 14)
(29, 14)


In [4]:
query_string='''
SELECT t1.installer_store_num, t1.store_num, t1.company_owned_store_ind, t1.state, t1.zip_cd, t2.latitude, t2.longitude  from mdb.vw__installer_master as t1
left join mdb_cdl.store t2 on t1.store_num=t2.store_num
where (open_store_flag='Y' or open_store_flag is null and company_owned_store_ind="Y") or (company_owned_store_ind="N" and t1.close_date is null);
'''

print("query: \n%s"%query_string)
df_all_open_stores=client.query(query_string).result().to_dataframe()
print("df_all_open_stores.shape",df_all_open_stores.shape)
df_all_open_stores['store_num']=df_all_open_stores['store_num'].astype(int).astype(str)
df_all_open_stores['zip_cd']=df_all_open_stores['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
print(df_all_open_stores.shape,df_all_open_stores['installer_store_num'].nunique())

df_all_open_stores.head(2)

query: 

SELECT t1.installer_store_num, t1.store_num, t1.company_owned_store_ind, t1.state, t1.zip_cd, t2.latitude, t2.longitude  from mdb.vw__installer_master as t1
left join mdb_cdl.store t2 on t1.store_num=t2.store_num
where (open_store_flag='Y' or open_store_flag is null and company_owned_store_ind="Y") or (company_owned_store_ind="N" and t1.close_date is null);



/home/jliang/.local/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


df_all_open_stores.shape (4608, 7)
(4608, 7) 4608


,installer_store_num,store_num,company_owned_store_ind,state,zip_cd,latitude,longitude
0,0000136383,6383,N,AL,35083,NaN,NaN
1,0000138004,8004,N,FL,32456,NaN,NaN


In [5]:
df_pair_dist_long=pd.DataFrame()
# Coorperate use store_id; Non-coorperate use installer_store_id

for ta, df_row in df_TA.groupby("TA_num"):
    P_zip_list=eval(df_row['trans_P_zips'].tolist()[0])
    P_zip_list=[str(x).zfill(5) for x in P_zip_list]
    df_stores_in_P=df_all_open_stores[df_all_open_stores['zip_cd'].isin(P_zip_list)]
    df_stores_in_P.reset_index(inplace=True)
    del df_stores_in_P['index']
    for i in range(len(df_stores_in_P)):
        if df_stores_in_P.loc[i,'company_owned_store_ind']=="Y":
            store_i_type="Cooperate"
            store_i_latlng=(df_stores_in_P.loc[i,'latitude'],df_stores_in_P.loc[i,'longitude'])
            ID_i=df_stores_in_P.loc[i,'store_num']
        else:
            store_i_type="Noncoop"
            store_i_latlng=zip_centers[df_stores_in_P.loc[i,'zip_cd']]
            ID_i=df_stores_in_P.loc[i,'installer_store_num']
            
        for j in range(len(df_stores_in_P)):
            if df_stores_in_P.loc[j,'company_owned_store_ind']=="Y":
                store_j_type="Cooperate"
                store_j_latlng=(df_stores_in_P.loc[j,'latitude'],df_stores_in_P.loc[j,'longitude'])
                ID_j=df_stores_in_P.loc[j,'store_num']
            else:
                store_j_type="Noncoop"
                store_j_latlng=zip_centers[df_stores_in_P.loc[j,'zip_cd']]
                ID_j=df_stores_in_P.loc[j,'installer_store_num']

            dist=haversine(store_i_latlng,store_j_latlng,unit="mi")
            
            df_pair_dist=pd.DataFrame({'Store_id_1':ID_i,'Store_id_2':ID_j,'dist':dist},index=[ta])
            df_pair_dist_long=df_pair_dist_long.append(df_pair_dist)
df_pair_dist_long=df_pair_dist_long.reset_index().rename(columns={"index":"TA_num"})
print(df_pair_dist_long.shape)

(474, 4)


In [6]:
df_pair_dist_long['Store_formatted_id_1']=np.where(df_pair_dist_long['Store_id_1'].apply(len)>=5,"TA: "+df_pair_dist_long['TA_num']+" | Noncoor | "+df_pair_dist_long['Store_id_1'],"TA: "+df_pair_dist_long['TA_num']+" | Coor | "+df_pair_dist_long['Store_id_1'])
df_pair_dist_long['Store_formatted_id_2']=np.where(df_pair_dist_long['Store_id_2'].apply(len)>=5,"TA: "+df_pair_dist_long['TA_num']+" | Noncoor | "+df_pair_dist_long['Store_id_2'],"TA: "+df_pair_dist_long['TA_num']+" | Coor | "+df_pair_dist_long['Store_id_2'])
df_pair_dist_long=df_pair_dist_long.sort_values(['TA_num','Store_formatted_id_1','Store_formatted_id_2'])

list_sorted_all_cols=df_pair_dist_long['Store_formatted_id_1'].unique().tolist()
list_sorted_all_cols.sort()
len(list_sorted_all_cols)

106

In [7]:
len(list_sorted_all_cols)

106

In [8]:
list_df_to_concat=[]
counter_i=0
df_output=pd.DataFrame({"index":list_sorted_all_cols},index=list_sorted_all_cols)
for TA_num,df_group in df_pair_dist_long.groupby("TA_num"):
    df=df_group.pivot_table(index=['TA_num','Store_formatted_id_1'],columns=['Store_formatted_id_2'],values=['dist'])
    df.columns=[x[1] for x in df.columns.tolist()]
    df.index=[x[1] for x in df.index.tolist()]
    df=df.reset_index()
    
    store_first=df['index'][0]
    store_last=df['index'].tolist()[-1]
    # print(df.shape)
    if store_first==list_sorted_all_cols[0]:
        df_remaining=pd.DataFrame(columns=df.columns.tolist()[1:],index=list_sorted_all_cols[list_sorted_all_cols.index(store_last)+1:]).reset_index()
        df_to_concat=df.append(df_remaining)
    elif store_last==list_sorted_all_cols[1]:
        df_heading=pd.DataFrame(columns=df.columns.tolist()[1:],index=list_sorted_all_cols[:list_sorted_all_cols.index(store_first)]).reset_index()
        df_to_concat=df_heading.append(df)
    else:
        df_heading=pd.DataFrame(columns=df.columns.tolist()[1:],index=list_sorted_all_cols[:list_sorted_all_cols.index(store_first)]).reset_index()
        df_remaining=pd.DataFrame(columns=df.columns.tolist()[1:],index=list_sorted_all_cols[list_sorted_all_cols.index(store_last)+1:]).reset_index()
        df_to_concat=df_heading.append(df).append(df_remaining)
    # list_df_to_concat.append(df_to_concat)
    # print(TA_num,df_to_concat.shape,df_to_concat['index'].nunique())
    counter_i+=1
    df_output=pd.merge(df_output,df_to_concat,on=["index"],how="left")


In [9]:
writer=pd.ExcelWriter("./GY_store_matrix_in_P_2to5_TAs_JL_%s.xlsx"%str(datetime.datetime.now().date()),engine="xlsxwriter")
df_output.to_excel(writer,"selected_maxtrix_wide",index=False)
df_pair_dist_long.to_excel(writer,"selected_maxtrix_long",index=False)
writer.save()